# Chapter 7: Classes and Interfaces

# Item 48: Accept Functions Instead of Classes for Simple Interfaces
Sometimes a class is too much man. Python treats functions as first-class citizens, and sometimes passing a function is enough.

Python's `list` has a sort method that takes a function, behold

In [2]:
names = ["Jimmy", "Andrew", "Luke", "Blake"]
names.sort(key=len)
names

['Luke', 'Jimmy', 'Blake', 'Andrew']

We passed the `len` function to the interface. This told the list to sort from shortest to longest name!

In Java we would have to make a class to do this same thing:
```java
class SortByLength implements Comparator<String> 
{    
    public int compare(String a, String b) {
        return a.length() < b.length();
    }
}

// OR

names.sort((first, second) -> Integer.compare(first.length(), second.length())
```

### Hooks
this type of interaction in programming is called a "hook"

In [4]:
# I have a defaultdict, but I want to emit a log message every time a missing key is encountered. Behold the power of HOOKS!

def log_missing():
    print("key added!")
    return 0


from collections import defaultdict

some_dict = {"green": 12, "blue": 3}

defaulted_dict = defaultdict(log_missing, some_dict)

defaulted_dict["green"] += 3
defaulted_dict["red"] += 5
defaulted_dict["blue"] += 7
dict(defaulted_dict)

key added!


{'green': 15, 'blue': 10, 'red': 5}

### But why do this?
This basically gives you a clean way to trigger side-effects and keep them separate from your deterministic behavior.

In [7]:
missed = 0


def count_missing():
    global missed  # remember nonlocal?
    missed += 1
    return 0


dict2 = defaultdict(count_missing, some_dict)
dict2["green"] += 3
dict2["orange"] += 2
dict2["purple"] += 1

print(missed)

2


Anyway, this is the power of first-class functions. Without this, you'd have to modify/extend `defaultdict` to get this behavior.

Lets do that example again, but without the gross `global` for state management.

In [9]:
class Counterater:
    def __init__(self):
        self.missed = 0

    def __call__(self):
        self.missed += 1
        return 0


my_cool_counter = Counterater()

dict2 = defaultdict(my_cool_counter,
                    some_dict)  # though I used __call__, you can also pass a class instance's method too.
dict2["green"] += 3
dict2["orange"] += 2
dict2["purple"] += 1

my_cool_counter.missed

2

### SURPRISE QUIZ: wtf is `__call__`?!
![spongebob.png](spongebob.png)

### Remember
Python functions are first-class, and thus allow composition, chaining, hooks, etc. Though a class to handle all the above may be a natural instinct, its not necessary in python.

# Item 49: Prefer Object-Oriented Polymorphism over Functions with `isinstance` Checks
TODO: sassy comment

In [12]:
# IN THIS EXAMPLE: building an AST for a pocket calculator
class Integer:
    def __init__(self, value):
        self.value = value


class Add:
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Multiply:
    def __init__(self, left, right):
        self.left = left
        self.right = right


tree = Add(Integer(1), Integer(2))

In [14]:
# We've got a tree... now what?
# EVAL THAT TREE BABY

def evaluate(node):
    if isinstance(node, Integer):
        return node.value
    elif isinstance(node, Add):
        return evaluate(node.left) + evaluate(node.right)
    elif isinstance(node, Multiply):
        return evaluate(node.left) * evaluate(node.right)
    else:
        raise NotImplementedError

In [18]:
print(evaluate(tree))
print(evaluate(Add(Integer(11), Integer(22))))
print(evaluate(
    Add(Multiply(Integer(2), Integer(2)), Integer(22))))  # it even supports arbitrary nesting, since it's recursive!

3
33
26


Works great! But imagine we wanted to make our AST much bigger than just Add and Multiply? Gosh that `if..elif` structure would get pretty huge...

Let's make this more OOP-y

In [19]:
class Node:
    def evaluate(self):
        raise NotImplementedError


class IntegerNode(Node):
    def __init__(self, value):
        self.value = value

    def evaluate(self):
        return self.value


class AddNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self):
        return self.left.evaluate() + self.right.evaluate()


class MultiplyNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self):
        return self.left.evaluate() * self.right.evaluate()

In [21]:
tree = AddNode(IntegerNode(1), IntegerNode(2))
tree.evaluate()

3

This allows polymorphic extension later via abstracts etc. Though, it's easy for this to become its own maintenance nightmare. Care and consideration should be had!

# Item 50: Consider `functools.singledispatch` for Functional-Style Programming Instead of Polymorphism

In [22]:
# Time to add pretty printing to that calculator!

class NodeAlt:
    def evaluate(self):
        raise NotImplementedError

    def pretty(self):
        raise NotImplementedError


class IntegerNodeAlt(NodeAlt):
    def __init__(self, value):
        self.value = value

    def evaluate(self):
        return self.value

    def pretty(self):
        return repr(self.value)


class AddNodeAlt(NodeAlt):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self):
        return self.left.evaluate() + self.right.evaluate()

    def pretty(self):
        left_str = self.left.pretty()
        right_str = self.right.pretty()
        return f"({left_str} + {right_str})"


class MultiplyNodeAlt(NodeAlt):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self):
        return self.left.evaluate() * self.right.evaluate()

    def pretty(self):
        left_str = self.left.pretty()
        right_str = self.right.pretty()
        return f"({left_str} * {right_str})"

In [24]:
tree = MultiplyNodeAlt(
    AddNodeAlt(IntegerNodeAlt(3), IntegerNodeAlt(5)),
    AddNodeAlt(IntegerNodeAlt(4), IntegerNodeAlt(7)),
)
tree.pretty()

'((3 + 5) * (4 + 7))'

Do you see where this can go wrong? What If I need to add 30 different methods to `NodeAlt`? I have to update EVERY SINGLE INHERITOR! BOO! That sucks!!!

As a fun aside: Effective Java also talks about this exact problem, and urges programmers to use `abstract` sparringly.

In [31]:
# Single Dispatch to the rescue!
from functools import singledispatch


@singledispatch
def pretty(value):
    raise NotImplementedError

Single-dispatch is a functional technique that lets you basically add behavior to a class without modifying it. The function above will be treated as the function of last resort, if nothing else can be found.

In [32]:
@pretty.register(int)
def _(value):  # note the name, _ basically means "the name doesn't matter"
    return repr(value)


@pretty.register(float)
def _(value):
    return f"{value:.2f}"

In [33]:
print(pretty(3.141592653589793))
print(pretty(3))

3.14
3


In [35]:
# Lets redo the AST

@singledispatch
def evaluate(node):
    raise NotImplementedError


class Integer:
    def __init__(self, value):
        self.value = value


class BinaryOp:
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Add(BinaryOp):
    def __init__(self, left, right):
        super().__init__(left, right)


class Multiply(BinaryOp):
    def __init__(self, left, right):
        super().__init__(left, right)


@evaluate.register(Integer)
def _(node):
    return node.value


@evaluate.register(Add)
def _(node):
    return evaluate(node.left) + evaluate(node.right)


@evaluate.register(Multiply)
def _(node):
    return evaluate(node.left) * evaluate(node.right)

# HEY DONT FORGET PRETTY~
@pretty.register(Integer)
def _(node):
    return repr(node.value)

@pretty.register(Add)
def _(node):
    left_str = pretty(node.left)
    right_str = pretty(node.right)
    return f"({left_str} + {right_str})"

@pretty.register(Multiply)
def _(node):
    left_str = pretty(node.left)
    right_str = pretty(node.right)
    return f"({left_str} * {right_str})"

In [36]:
tree = Multiply(Add(Integer(1), Integer(2)), Add(Integer(3), Integer(4)))
print(evaluate(tree))
print(pretty(tree))

21
((1 + 2) * (3 + 4))


An immediate benefit of this, is how the dispatch works with sub-classing.

In [37]:
class PositiveInteger(Integer):
    def __init__(self, value):
        if value < 0:
            raise ValueError("value must be positive")
        super().__init__(value)

pretty(PositiveInteger(3))

'3'

However, things get harder when we create a brand new class

In [38]:
class Float:
    def __init__(self, value):
        self.value = value

pretty(Float(3.141592653589793))

NotImplementedError: 

This is a trade-off.
- Polymorphism: easy to add new classes, PITA to add new behavior across classes.
- Single Dispatch: easy to add new behavior across classes, PITA to add new classes.

From experience, the former is the *bigger* PITA. Single dispatch allows classes themselves to remain much simpler, and allows you as the programmer, to have independent systems with their own behaviors, use those classes without polluting them.

To solidify: look at Video Center and `abstract`; this is polymorphism, and it's awful. A single change causes cascading maintenance headaches everywhere. As a result, changes are seldom made, and the code-base has aged horribly.

But polymorphism still makes sense in smaller contexts, or fairly self-contained contexts, where the classes don't need to be shared across system boundaries.

# Item 51: Prefer `dataclasses` for Defining Lightweight Classes

In [51]:
# dataclasses are pretty cool
from dataclasses import dataclass

@dataclass
class MyRGB:
    red: int
    green: int
    blue: int

# They enforce order
color1 = MyRGB(1, 2, 3)
color2 = MyRGB(red=1, green=2, blue=3)

# They allow type checking! (a linter would catch this)
color1.red = "two"

In [55]:
# heck we can force the user to name their args
@dataclass(kw_only=True)
class MyRGB:
    red: int
    green: int
    blue: int

MyRGB(1, 2, 3)

TypeError: MyRGB.__init__() takes 1 positional argument but 4 were given

In [54]:
MyRGB(red=1, green=2, blue=3)

MyRGB(red=1, green=2, blue=3)

In [60]:
# We also get functioning comparisons!
print(color1 == color2)
print(MyRGB(red=1, green=2, blue=3) == MyRGB(red=1, green=2, blue=3))

False
True


In [68]:
# Want a dictionary? You got it pal
from dataclasses import asdict
asdict(color1)

{'red': 1, 'green': 2, 'blue': 3}

In [62]:
# Complicated ordering
class Planet:
    def __init__(self, distance, size):
        self.distance = distance
        self.size = size

    def __repr__(self):
        return (
            f"{type(self).__module__}"
            f"{type(self).__name__}("
            f"distance={self.distance}, "
            f"size={self.size})"
        )

far = Planet(10, 5) # planet size 5, 10 away from us
near = Planet(1, 2) # planet size 2, 1 away from us
data = [far, near]
# This is going to fail because python doesn't know how to compare planets!
data.sort()

TypeError: '<' not supported between instances of 'Planet' and 'Planet'

In [63]:
# In order to make them comparable, we'll need to add the _astuple helper
class Planet:
    ...

    def _astuple(self):
        return (self.distance, self.size)

    def __eq__(self, other):
        return (
            type(self) == type(other)
            and self._astuple() == other._astuple()
        )

    def __lt__(self, other):
        if type(self) != type(other):
            return NotImplemented
        return self._astuple() < other._astuple()

    def __le__(self, other):
        if type(self) != type(other):
            return NotImplemented
        return self._astuple() <= other._astuple()

    def __gt__(self, other):
        if type(self) != type(other):
            return NotImplemented
        return self._astuple() > other._astuple()

    def __ge__(self, other):
        if type(self) != type(other):
            return NotImplemented
        return self._astuple() >= other._astuple()

In [66]:
# Oh boy.. thats a lot.. Dataclasses you got anything for us?!
@dataclass(order=True)
class DataclassPlanet:
    distance: float
    size: float

far2 = DataclassPlanet(10, 2)
near2 = DataclassPlanet(1, 5)
print(far2 > near2)
print(near2 < far2)

data = [far2, near2]
data.sort()
data

True
True


[DataclassPlanet(distance=1, size=5), DataclassPlanet(distance=10, size=2)]

As you can see, `dataclass` is _pretty cool_. A possible Java equivalent might be Lombok's `Data` annotation.

Think of it as a boilerplate slayer.

In [69]:
# What about Pydantic?
# Pydantic is good stuff. Let's see how it compares to dataclasses.

import sys
from pydantic import BaseModel

@dataclass
class PointV1:
    x: float
    y: float

class PointV2(BaseModel):
    x: float
    y: float

point1 = PointV1(1, 2)
point2 = PointV2(x=1, y=2)
print("The dataclass is", sys.getsizeof(point1), "bytes")
print("The pydantic model is", sys.getsizeof(point2), "bytes")

The dataclass is 48 bytes
The pydantic model is 72 bytes


In [45]:
# Why is pydantic bigger? Well there's just a lot more to a Pydntic model than a dataclass. DIR EM:
print(dir(point1))
print('-' * 80)
print(dir(point2))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__replace__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', 'x', 'y']
--------------------------------------------------------------------------------
['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__firstlineno__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__'

This kind of difference can compound at scale. Prefer dataclasses to pydantic where you _do not_ need pydantic's powerful validation tooling.

By the way: `dir` is just a cool lil function that either dumps the names in the current local scope, OR lists out all the attributes of an object. I like to use it as a fast and easy way to discover methods and fields on an object.

In [70]:
# Pydantic, of course, gives us some pretty strong RUNTIME validation

PointV2(x=1, y="two")  # dataclass would allow this at runtime

ValidationError: 1 validation error for PointV2
y
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='two', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing

# Item 52: Use `@classmethod` Polymorphism to Construct Objects Genericall